In [1]:
from langchain.chains import ConversationalRetrievalChain
from langchain_anthropic import ChatAnthropic
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

/home/kelechi/code/Machine_Learning/web_chat/.lc_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", google_api_key=os.getenv("GOOGLE_API_KEY")
)

# Load documents
loader = WebBaseLoader("https://www.anthropic.com/claude")
documents = loader.load()


# Create vector store
vector_store = Chroma.from_documents(documents, embeddings)

# Initialize conversation memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize language model
llm = ChatAnthropic(
    anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"),
    model="claude-3-sonnet-20240229",
    temperature=0.2,
    max_tokens=1024,
)

# Create the chain
qa = ConversationalRetrievalChain.from_llm(
    llm, vector_store.as_retriever(), memory=memory
)

In [5]:
# Start the conversation
chat_history = []
query = "What is Anthropics lightest model?"
result = qa.invoke({"question": query, "chat_history": chat_history})
print(result["answer"])



ValueError: Unsupported chat history format: <class 'str'>. Full chat history: Human: What is Anthropics lightest model?
AI: According to the information provided, Anthropic's lightest and fastest model in the Claude family is called Haiku.

The context states:

"The Claude 3 family of models offers the best combination of speed and performance for enterprise use cases, at a lower cost than other models on the market.

Light & fast
Haiku
Our fastest model that can execute lightweight actions, with industry-leading speed."

So Haiku is described as Anthropic's fastest and lightest model for lightweight tasks requiring high speed. 

In [4]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS, Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


from dotenv import load_dotenv
import os  # type: ignore

# st.set_page_config(page_title="Web Chat", page_icon=":rocket:")


load_dotenv()

base_llm = ChatAnthropic(
    anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"),
    model="claude-3-sonnet-20240229",
    temperature=0.2,
    max_tokens=1024,
)

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the the provided context:
     <context>
         {context}
     </context>

     Question: {input}
    """
)

document_chain = create_stuff_documents_chain(base_llm, prompt)

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", google_api_key=os.getenv("GOOGLE_API_KEY")
)

text_splitter = RecursiveCharacterTextSplitter()


def load_website_data(url):
    loader = WebBaseLoader(url)
    docs = loader.load()
    documents = text_splitter.split_documents(docs)
    vector = FAISS.from_documents(documents, embeddings)
    retriever = vector.as_retriever()

    return retriever


# loader = WebBaseLoader("https://www.anthropic.com/claude")
# input_url = st.text_input("URL: ")


# retriever = load_website_data('https://www.anthropic.com/claude')
retriever = load_website_data("https://en.wikipedia.org/wiki/Perplexity.ai")

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [5]:
query = "What are the main infromations on the site?"
result = retrieval_chain.invoke({"input": query})

print(result["answer"])

According to the Wikipedia article, the main information provided about Perplexity.ai is:

1. It is an AI-powered research/conversational search engine that answers queries using natural language predictive text.

2. It was launched in 2022 and generates answers using sources from the web, citing links within the text responses.

3. It has a freemium model - the free version uses Perplexity's own model based on GPT-3.5, while the paid "Perplexity Pro" version has access to GPT-4, Claude 3, and other large language models.

4. It was founded in August 2022 by Andy Konwinski, Denis Yarats, Johnny Ho, and Aravind Srinivas (the current CEO who previously worked at OpenAI).

5. As of 2024, it has raised $100 million in funding valuing the company at $520 million, with investors including Jeff Bezos, Nvidia, Databricks and others.

6. Its main functionality is providing personalized search results by interpreting the context of user queries and summarizing results with inline citations, prio